<font size=5> Global settings and imports<font>

In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
import pickle
from Hypers import *
from Preprocessing import *
from Generator import *
from Model import *
from Utils import *

In [4]:
data_path = 'Data'
embedding_path = 'Data/utils'
pre_model_file = 'pre_model/preModel'

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(
    data_path, 'train', r'train_IID_behaviors.tsv')
valid_news_file = os.path.join(data_path, 'train', r'news.tsv')
valid_behaviors_file = os.path.join(
    data_path, 'train', r'valid_IID_behaviors.tsv')

test_news_file = os.path.join(data_path, 'test', r'news.tsv')
test_behaviors_file = os.path.join(
    data_path, 'test', r'test_OOD_behaviors.tsv')
test_behaviors_file1 = os.path.join(
    data_path, 'test', r'test_IID_behaviors.tsv')

pop_path = os.path.join(data_path, 'utils', r'news_pop.pkl')

<font size =5>Preprocessing data<font>

In [5]:
with open(pop_path, 'rb') as fp:
    news_pop_cnt = pickle.load(fp)

pop_max = len(set(news_pop_cnt.values()))

news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict = read_news(
    [train_news_file, test_news_file])

title_word_embedding_matrix, have_word = load_matrix(embedding_path,word_dict)
word_embedding_matrix = title_word_embedding_matrix
vocab_size = word_embedding_matrix.shape[0]

news_title, news_vert, news_subvert, news_entity, news_content, news_pop, tar_title_inp, tar_title_real = get_doc_input(
    news, news_index, category_dict, subcategory_dict, entity_dict, word_dict,content_dict, news_pop_cnt, pop_max, True, DROPOUT_RATE)

news_info = np.concatenate([news_title, news_pop.reshape((-1, 1))], axis=-1)
news_vae_info = np.concatenate(
    [news_title, tar_title_inp, news_pop.reshape((-1, 1)), tar_title_real], axis=-1)

<font size=5>Train the CDNR model<font>

In [ ]:
train_session_all = read_train_clickhistory(
    news_index, train_behaviors_file, FILTER_NUM)
train_session = train_session_all
train_user = parse_user(news_index, train_session)
train_sess, train_user_id, train_label = get_train_input(
    news_index, train_session)

train_generator = get_hir_train_generator(
    news_info, train_user['click'], train_user_id, train_sess, train_label, BATCH_SIZE, news_vae_info, MAX_VAE_TITLE)

test_session_all_OOD = read_test_clickhistory(
    news_index, test_behaviors_file, FILTER_NUM)
test_session_OOD = test_session_all_OOD
test_user_OOD = parse_user(news_index, test_session_OOD)
test_impressions_OOD, test_userids_OOD = get_test_input(
    news_index, test_session_OOD)

test_session_all_IID = read_test_clickhistory(
    news_index, test_behaviors_file1,FILTER_NUM)

test_session_IID = test_session_all_IID
test_user_IID = parse_user(news_index, test_session_IID)
test_impressions_IID, test_userids_IID = get_test_input(
    news_index, test_session_IID)

model, news_encoder, model1, user_encoder, trans = create_model(
    word_embedding_matrix, NUM_LAYERS, D_MODEL, D_LATENT, NUM_HEADS, True, DROPOUT_RATE, W_VAE, 0.0, W_VAE, MAX_POSITION, pop_max)
model.fit(train_generator, epochs=2)
model.save_weights(pre_model_file)

model, news_encoder, model1, user_encoder, trans = create_model(
    word_embedding_matrix, NUM_LAYERS, D_MODEL, D_LATENT, NUM_HEADS, True, DROPOUT_RATE, W_VAE, W_VAE/2.0, W_VAE, MAX_POSITION, pop_max)

model.load_weights(pre_model_file)
model.fit(train_generator, epochs=EPOCHS)

<font size=5>Evaluate the CDNR model<font>

In [ ]:
news_scoring = news_encoder.predict(news_info, verbose=1)
test_generator_OOD = get_user_generator(
    news_info, test_userids_OOD, test_user_OOD['click'], BATCH_SIZE)
test_user_scoring_OOD = user_encoder.predict_generator(
    test_generator_OOD, verbose=1)
AUC_OOD, MRR_OOD, nDCG5_OOD, nDCG10_OOD = evaluate(
    test_impressions_OOD, news_scoring[:, :D_LATENT], test_user_scoring_OOD[:, :D_LATENT])

test_generator_IID = get_user_generator(
    news_info, test_userids_IID, test_user_IID['click'], BATCH_SIZE)
test_user_scoring_IID = user_encoder.predict_generator(
    test_generator_IID, verbose=1)
AUC_IID, MRR_IID, nDCG5_IID, nDCG10_IID = evaluate(
    test_impressions_IID, news_scoring, test_user_scoring_IID)

In [ ]:
print('OOD - ','AUC:',AUC_OOD,' MRR:', MRR_OOD, ' nDCG5:', nDCG5_OOD, ' nDCG10:', nDCG10_OOD)
print('IID - ','AUC:',AUC_IID,' MRR:', MRR_IID, ' nDCG5:', nDCG5_IID, ' nDCG10:', nDCG10_IID)